## Exercise 4

For this exercise you are given a database containing information about real estate around
Uppsala. You are also given a Python module that helps working the database.

- Download the [database](...) and the [code `db.py`](...)
- Read the module's [documentation](../../files/db_module_documentation.html)

The module `db.py` will help you produce a HTML file whit your results plotted on a map. This set-up uses OpenStreetMap. 

![result](../../img/mapOSM.jpg)

The center is defined as the point:

`latitude: 59.865795990339876`

`longitude: 17.64583576202392`

### Questions:


a) What is the price of the cheapest house around the given center and within 2 kilometers?

b) Plot the 100 cheapest houses in this area.

c) Find the most expensive house per square meter and print it's price and square meter. If a house have an area of 0 square meters, discard it!

d) Print or plot this house.